# <center>Car Maker Classification (VGG16 Keras)</center>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

from keras.callbacks import ModelCheckpoint

In [2]:
keras.__version__

'2.7.0'

## Run on GPU

In [3]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# print("Num GPUs Available: ", len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Preparing images

In [4]:
train_path = '../dataset/train_spl'
valid_path = '../dataset/val_spl'
test_path = '../dataset/test_spl'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 9791 images belonging to 196 classes.
Found 3312 images belonging to 196 classes.
Found 3082 images belonging to 196 classes.


## Choosing Model

In [5]:
model_original = tf.keras.applications.vgg16.VGG16()

In [6]:
# model_original.summary() # to show all models layers

In [7]:
# taking all layers except last 6 layers
x = model_original.layers[-2].output
output = Dense(units=196, activation='softmax')(x)

In [8]:
model = Model(inputs=model_original.input, outputs=output)

In [9]:
# freezing last n layers (n is an arbitrary number)
# for layer in model.layers:
#     layer.trainable = False

In [10]:
# model.summary() # to show all models layers

## Training Model

In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# makeing checkpoint each epoch
checkpoint = ModelCheckpoint("M_VGG1_checkpoints/M_VGG1_checkpoint.hdf5", monitor='loss', verbose=0,
    save_best_only=False, mode='auto', save_freq=1)

In [ ]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=30,
            verbose=2,
            callbacks=[checkpoint]
)

Epoch 1/30


## Saving Model

In [ ]:
model.save("M_VGG16.h5")

## Result Visualization

In [ ]:
test_labels = test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

In [ ]:
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     plt.rcParams["figure.figsize"] = (12,12)
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#             horizontalalignment="center",
#             color="white" if cm[i, j] > thresh else "black")

In [ ]:
# cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

In [ ]:
# test_batches.class_indices

In [ ]:
# cm_plot_labels = list(range(0, 196))
# plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')